# Blue Rockfish Habitat Predictions

In [1]:
import os
import sys
import numpy as np
import matplotlib
import pandas as pd
import geopandas as gpd
import json
import pickle
import xgboost
import torch
import pickle
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from simpledbf import Dbf5

In [3]:
def preprocess_eval_data(files):
    pass

In [ ]:
def preprocess_train_data(files):
    """
    description
    
    param files: 
    """
    
    # set up env
    arcpy.env.workspace = 'data'
    arcpy.env.overwriteOutput = True
    save_path = '../../scratch/'
    
    # gather the shape files
    files = np.array(files)
    obs = files[0]
    rnd = files[1]
    
    # gather the raster files
    rasters = files[2:]
    rasters = list(rasters[rasters != '#'])

    # sample rasters from observed points
    obs_table = save_path + 'observed_presence_sampled.shp'
    obs_points = arcpy.sa.Sample(rasters, obs, obs_table)
    obs_count = int(arcpy.management.GetCount(obs_table)[0])
    obs_label = np.array([1]*obs_count)
    
    # sample rasters from absence points
    rnd_table = save_path + 'random_absence_sampled.shp'
    rnd_points = arcpy.sa.Sample(rasters, rnd, rnd_table)
    rnd_count = int(arcpy.management.GetCount(rnd_table)[0])
    rnd_label = np.array([0]*rnd_count)

    return obs_table, obs_label, rnd_table, rnd_label

In [51]:
def build_model(files):
    """
    Description
    
    param files: 
    """
    # preprocess data
    obs_path, obs_label, rnd_path, rnd_label = files
    
    # read training file
    obs_dbf = Dbf5(obs_path)
    rnd_dbf = Dbf5(rnd_path)
    obs_df = obs_dbf.to_dataframe()
    rnd_df = rnd_dbf.to_dataframe()

    # prepare data for training
    obs_df = obs_df.drop(columns = ['brf_obs', 'X', 'Y'])
    rnd_df = rnd_df.drop(columns = ['rand_obs', 'X', 'Y'])
    obs_df['label'] = obs_label
    rnd_df['label'] = rnd_label
    data = pd.concat([obs_df, rnd_df])
    X = data.drop(columns = ['label'])
    Y = data['label']
    
    # build xgboost model
    clf = XGBClassifier(objective= 'binary:logistic')
    parameters = {
        'max_depth': range (2, 10),
        'n_estimators': range(60, 220, 40),
        'learning_rate': [0.1, 0.01, 0.05]
    }
    grid_search = GridSearchCV(
        estimator = clf,
        param_grid = parameters,
        scoring = 'roc_auc',
        n_jobs = 5,
        cv = 5,
        verbose = True)
    grid_search.fit(X, Y)
    
    model_path = "../../scratch/xgb_model.pkl"
    pickle.dump(grid_search, open(model_path, "wb"))

In [ ]:
def make_prediction(files):
    """
    Description
    
    param files
    """
    files = files[0]
    
    # process files for prediction
    data = preprocess_eval_data(files)
    
    # read model
    model = pickle.load(open('../../scratch/xgb_model.pkl', 'rb'))
    
    # save raster file

### Run Training Pipeline

In [ ]:
training_input_files = []
build_model(input_files)

### Run Testing Pipeline

In [ ]:
testing_input_files = []
raster = make_prediction(input_files)